In [1]:
pip install facebook-scraper

In [2]:
from facebook_scraper import get_posts
for post in get_posts('PedroPalaciosU', pages=1):
    print(post)
    #print(post["likes"])  
    #print(post['comments_full'])
    #print(post['text'])
    #print(post['images'])

C:\Users\DomenicaMunoz\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3418: UserWarning: A low page limit (<=2) might return no results, try increasing the limit
  exec(code_obj, self.user_global_ns, self.user_ns)
C:\Users\DomenicaMunoz\anaconda3\lib\site-packages\facebook_scraper\facebook_scraper.py:235: UserWarning: Locale detected as es_LA - for best results, set to en_US
  warnings.warn(f"Locale detected as {locale} - for best results, set to en_US")


{'post_id': '790687974983155', 'text': '¡La economía en Cuenca se reactiva!\nSuscribimos el contrato de 48 millones de dólares con el Banco de Desarrollo para América Latina para el financiamiento e implementación de obras que permitirán crear plazas… Más de trabajos directas e indirectas y el desarrollo social de la ciudad.\nLos recursos económicos se destinarán para programas y proyectos que beneficien a las familias cuencanas con obras, fiscalización, consultorías, estudios, manejo ambiental y social, auditorías, bienes y fortalecimiento institucional.\n#CuencaUnida\n\n¡La economía en Cuenca se reactiva!\n\nSuscribimos el contrato de 48 millones de dólares con el Banco de Desarrollo para América Latina para el financiamiento e implementación de obras que permitirán crear plazasMás de trabajos directas e indirectas y el desarrollo social de la ciudad.\n\nLos recursos económicos se destinarán para programas y proyectos que beneficien a las familias cuencanas con obras, fiscalización, 

In [3]:
pip install neo4j

Note: you may need to restart the kernel to use updated packages.


In [4]:
from neo4j import GraphDatabase 
class Neo4jService(object):

    def __init__ (self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))

    def close(self):
        self._driver.close()
    def crear_alcaldes(self, tx, nombre):
        tx.run("CREATE (:alcaldes {nombre: $nombre})", nombre=nombre)
    def crear_noticia(self, tx, nombre):
        tx.run("CREATE (:noticia {nombre: $nombre})", nombre=nombre)
    def crear_links(self, tx, nombre):
        tx.run("CREATE (:links {nombre: $nombre})", nombre=nombre)
        
    def crear_contenido(self, tx, nombre):
        tx.run("CREATE (:contenido {nombre: $nombre})", nombre=nombre)
        
    def crear_titulo(self, tx, nombre):
        tx.run("CREATE (:titulo {nombre: $nombre})", nombre=nombre)

    def crear_fecha(self, tx, nombre):
        tx.run("CREATE (:fecha {nombre: $nombre})", nombre=nombre)
    def crear_likes(self, tx, nombre):
        tx.run("CREATE (:likes {nombre: $nombre})", nombre=nombre)
    
    def crear_relacion_AN(self, tx, nombre_alcalde,nombre_noticia):
        tx.run("MATCH (a:alcaldes {nombre: $nombre_alcalde}) "
               "MATCH (b:noticia {nombre: $nombre_noticia}) "
               "MERGE (a)-[:alcalde_noticia]->(b)",
               nombre_alcalde=nombre_alcalde, nombre_noticia=nombre_noticia)
    def crear_relacion_AL(self, tx, nombre_alcalde,nombre_links):
        tx.run("MATCH (a:alcaldes {nombre: $nombre_alcalde}) "
               "MATCH (b:links {nombre: $nombre_links}) "
               "MERGE (a)-[:alcalde_links]->(b)",
               nombre_alcalde=nombre_alcalde, nombre_links=nombre_links)
    def crear_relacion_AT(self, tx, nombre_alcalde,nombre_titulo):
        tx.run("MATCH (a:alcaldes {nombre: $nombre_alcalde}) "
               "MATCH (b:titulo {nombre: $nombre_titulo}) "
               "MERGE (a)-[:alcalde_titulo]->(b)",
               nombre_alcalde=nombre_alcalde, nombre_titulo=nombre_titulo)
    def crear_relacion_AF(self, tx, nombre_alcalde,nombre_fecha):
        tx.run("MATCH (a:alcaldes {nombre: $nombre_alcalde}) "
               "MATCH (b:fecha {nombre: $nombre_fecha}) "
               "MERGE (a)-[:alcalde_fecha]->(b)",
               nombre_alcalde=nombre_alcalde, nombre_fecha=nombre_fecha)
    def crear_relacion_ALI(self, tx, nombre_alcalde,nombre_likes):
        tx.run("MATCH (a:alcaldes {nombre: $nombre_alcalde}) "
               "MATCH (b:likes {nombre: $nombre_likes}) "
               "MERGE (a)-[:alcalde_likes]->(b)",
               nombre_alcalde=nombre_alcalde, nombre_likes=nombre_likes)
    

In [ ]:
neo4j = Neo4jService('bolt://localhost:7687', 'neo4j', 'cuenca')
with neo4j._driver.session() as session:
    from facebook_scraper import get_posts
    session.write_transaction(neo4j.crear_alcaldes , "Pedro Palacios Ullauri")
for post in get_posts('PedroPalaciosU', pages=300):
    #print(post)
    #print(post["likes"])  
    #print(post['comments_full'])
    #print(post['text'])
    #print(post['images'])
    session.write_transaction(neo4j.crear_noticia , post['post_text'])
    session.write_transaction(neo4j.crear_links , post['user_url'])
    session.write_transaction(neo4j.crear_contenido , post['text'])
    session.write_transaction(neo4j.crear_likes , post['likes'])
    session.write_transaction(neo4j.crear_titulo , post['images_description'])
    session.write_transaction(neo4j.crear_fecha , post['time'])    
    session.write_transaction(neo4j.crear_relacion_AN , "Pedro Palacios Ullauri",post['post_text'])
    session.write_transaction(neo4j.crear_relacion_AL , "Pedro Palacios Ullauri",post['user_url'])
    session.write_transaction(neo4j.crear_relacion_AT , "Pedro Palacios Ullauri",post['images_description'])
    session.write_transaction(neo4j.crear_relacion_AF , "Pedro Palacios Ullauri",post['time'])
    session.write_transaction(neo4j.crear_relacion_ALI , "Pedro Palacios Ullauri",post['likes'])